## Introduction

### `numpy` limitations

`numpy` does not allow to:

- assign custom labels to data
- perform common database-like operations
- import/export easily data from the disk

### About `pandas`

`pandas` is a "data analysis and manipulation tool". In the backend, `pandas` relies on `numpy`, which makes it fast for many operations.

`pandas` presents 2 different data containers:

- `DataFrame`: similar to a 2D numpy array with:
    
    - rows and columns labels
    - possibly heterogeneous data
    
- `Series`: similar to a 1D numpy array

### `pandas` and notebooks

`pandas` objets have a pretty representation in notebooks: instead of printing them, just call them as the last statement of the cell.

## Dataframe

In [1]:
import pandas as pd
import numpy as np

Dataframes can be built in several ways. For instance, using a dictionary:

In [2]:
data = {'Some integers': (1, 2, 3), 'Some booleans': (True, False, True), 'Some strings': ('a', 'b', 'c')}
pd.DataFrame(data=data)

,Some integers,Some booleans,Some strings
0,1,True,a
1,2,False,b
2,3,True,c


Or an iterable:

In [3]:
data = ((1, True, 'a'), (2, False, 'c'), (3, True, 'c'))
columns = ('Some integers', 'Some booleans', 'Some strings')
pd.DataFrame(data=data, columns=columns)

,Some integers,Some booleans,Some strings
0,1,True,a
1,2,False,c
2,3,True,c


One can notice an additional columns on the left side: this is __the index along axis 0__, or more simply "index". Index along axis 1 is also called "columns".

One can specify the index values:

In [4]:
data = ((1, True, 'a'), (2, False, 'b'), (3, True, 'c'))
columns = ('Some integers', 'Some booleans', 'Some strings')
index = ('first row', 'second row', 'third row')
df = pd.DataFrame(data=data, columns=columns, index=index)
df

,Some integers,Some booleans,Some strings
first row,1,True,a
second row,2,False,b
third row,3,True,c


## Series

A `Series` object is similar to a `DataFrame` with one column. Instead of having 'columns', a `Series` has a `name` attribute.

In [5]:
data = (True, False, True)
pd.Series(data=data, name='Some booleans', index=index)

first row      True
second row    False
third row      True
Name: Some booleans, dtype: bool

## Access data

_note: explanations below are for `DataFrame`, but similar behaviour is observed for `Series`._

One can access data in a `DataFrame` in 2 ways:

- indexing: the same way one would do with a numpy array
- using labels

### Using indexing

The important method is __`iloc`__, which is used using brackets `[]`:

- first value selects along axis 0
- second value selects along axis 1


In [6]:
df

,Some integers,Some booleans,Some strings
first row,1,True,a
second row,2,False,b
third row,3,True,c


In [7]:
df.iloc[1, 2]

'b'

_slicing_ is also possible. Let's extract:

- One row out of two from 0 to 3
- The last two columns

In [8]:
df.iloc[0:3:2, -2:]

,Some booleans,Some strings
first row,True,a
third row,True,c


Another way is to specify directly a __list__ of indexes:

In [9]:
df.iloc[[0, 2], [-2, -1]]

,Some booleans,Some strings
first row,True,a
third row,True,c


Or a boolean indexer:

In [10]:
df.iloc[[True, False, True], [False, True, True]]

,Some booleans,Some strings
first row,True,a
third row,True,c


Beware! The type of returned object depend on the way indexing is done:

In [11]:
print(type(df.iloc[0:3:2, -1:]))    # every column from -1 to the end --> DataFrame

<class 'pandas.core.frame.DataFrame'>


In [12]:
print(type(df.iloc[0:3:2, -1]))    # specifically the last column --> Series

<class 'pandas.core.series.Series'>


In [13]:
print(type(df.iloc[0:3:2, [-1]]))    # the columns specified by one-element list --> DataFrame

<class 'pandas.core.frame.DataFrame'>


As with `numpy`, `:` is used to get all the data along a specific axis:

In [14]:
df.iloc[:, [-2, -1]]

,Some booleans,Some strings
first row,True,a
second row,False,b
third row,True,c


For columns (axis 1), one can also undefine the index in order to get all data.

In [15]:
df.iloc[[0, 2]]

,Some integers,Some booleans,Some strings
first row,1,True,a
third row,3,True,c


### Using labels

Similarly to `iloc`, `loc` allows to access elements using their labels:

In [16]:
df

,Some integers,Some booleans,Some strings
first row,1,True,a
second row,2,False,b
third row,3,True,c


In [17]:
df.loc['first row', 'Some strings']

'a'

In [18]:
df.loc[['first row', 'third row'], 'Some strings']

first row    a
third row    c
Name: Some strings, dtype: object

If axis=0 does not matter, simple brackets `[]` can be used to access columns. 

Hereafter, the two solutions are equivalent:

In [19]:
df.loc[:, ['Some booleans', 'Some strings']]
df[['Some booleans', 'Some strings']]

,Some booleans,Some strings
first row,True,a
second row,False,b
third row,True,c


Note: to modify a value, one must __always__ :

- use `loc` or `iloc` 
- specify the 2 coordinates in a single call.

    Else, a _warning_ is raised.

In [20]:
df.loc['first row', 'Some integers'] = 42
df

,Some integers,Some booleans,Some strings
first row,42,True,a
second row,2,False,b
third row,3,True,c


In [21]:
df.loc['third row'].iloc[2] = 'new_string'   # warning is raised
df

/tmp/ipykernel_73600/3522855576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc['third row'].iloc[2] = 'new_string'   # warning is raised


,Some integers,Some booleans,Some strings
first row,42,True,a
second row,2,False,b
third row,3,True,c


## Modify index


### From existing data

`set_index` method replaces the current index with a columns of the DataFrame and then:
    
- deletes the column if `drop` = True (defaut)
- keeps the column if `drop` = False
    

In [22]:
df

,Some integers,Some booleans,Some strings
first row,42,True,a
second row,2,False,b
third row,3,True,c


In [23]:
df.set_index('Some booleans', drop=False)

,Some integers,Some booleans,Some strings
Some booleans,,,
True,42,True,a
False,2,False,b
True,3,True,c


In [24]:
df.set_index('Some booleans')

,Some integers,Some strings
Some booleans,,
True,42,a
False,2,b
True,3,c


### Using integers

`reset_index` method replaces the current index by integers. It:

- deletes the current index if `drop` = True (defaut)
- keeps the current index as a column if `drop` = False

In [25]:
df.reset_index(drop=False)

,index,Some integers,Some booleans,Some strings
0,first row,42,True,a
1,second row,2,False,b
2,third row,3,True,c


Si l'index avait un nom, la colonne résultante porte son nom.

In [26]:
df.index.name = 'my_index'
df.reset_index(drop=False)

,my_index,Some integers,Some booleans,Some strings
0,first row,42,True,a
1,second row,2,False,b
2,third row,3,True,c


## Iteration

Iteration over a dataframe is pretty slow but still made possible using dedicated methods:

### Over rows

The `iterrows` method is used to iterate over rows, including index.

In [27]:
for index_value, (integer, string) in df[['Some integers', 'Some strings']].iterrows():
    print(index_value, integer, string)

first row 42 a
second row 2 b
third row 3 c


### Over columns

Let's use `itertuples`.At each iteration, this method returns:

- the column name
- the column data as a `Series` instance 



In [28]:
for column_name, column in df[['Some integers', 'Some strings']].items():
    print(column_name, '\n\n', column)

Some integers 

 my_index
first row     42
second row     2
third row      3
Name: Some integers, dtype: int64
Some strings 

 my_index
first row     a
second row    b
third row     c
Name: Some strings, dtype: object


## Basic operations


### Similar to `numpy`

__Many `numpy` operations are similar with `pandas`.__ 

In [29]:
data = np.arange(16).reshape((4, 4))
df = pd.DataFrame(data=data, columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [30]:
df.sum(axis=1)  # sum of the first 4 columns

0     6
1    22
2    38
3    54
dtype: int64

The `agg` method perform element-wise operations:

In [31]:
df['E'] = df['B'].agg(lambda x: x if x%5==0 else 42)
df

,A,B,C,D,E
0,0,1,2,3,42
1,4,5,6,7,5
2,8,9,10,11,42
3,12,13,14,15,42


Yet, the same result could be achieved using `where`.

In [32]:
cond = df['B']%5==0
df['E'] = df['B']                  # values of B if cond is True
df['E'] = df['E'].where(cond, 42)  # 42 if cond is False 
                                   # (opposite to numpy where 
                                   # first arg corresponds to positive cond)
df

,A,B,C,D,E
0,0,1,2,3,42
1,4,5,6,7,5
2,8,9,10,11,42
3,12,13,14,15,42


### Sorting data


One can use the `sort_values` and `sort_index` methods:

In [33]:
data = np.random.randint(0, 10, (5, 5))
df = pd.DataFrame(data=data, columns=('c1', 'c2', 'c3', 'c4', 'c5'), index=('e', 'b', 'c', 'a', 'd'))
df

,c1,c2,c3,c4,c5
e,6,5,4,3,8
b,6,1,8,8,9
c,3,2,4,0,9
a,9,7,2,4,4
d,9,9,8,0,5


In [34]:
df.sort_index()

,c1,c2,c3,c4,c5
a,9,7,2,4,4
b,6,1,8,8,9
c,3,2,4,0,9
d,9,9,8,0,5
e,6,5,4,3,8


Let's sort `df` according to:

- column `c2`
- descending order

In [35]:
df.sort_values(by='c2', ascending=False)

,c1,c2,c3,c4,c5
d,9,9,8,0,5
a,9,7,2,4,4
e,6,5,4,3,8
c,3,2,4,0,9
b,6,1,8,8,9


Let's sort following axis 1!

- row `b`
- custom key


In [36]:
# using `key`, the closest the values are to 5 the sooner 
# they come in the DataFrame
df.sort_values(by='b', axis=1, key=lambda x: abs(x-5))

,c1,c3,c4,c2,c5
e,6,4,3,5,8
b,6,8,8,1,9
c,3,4,0,2,9
a,9,2,4,7,4
d,9,8,0,9,5


## Manage _dtype_

You must __always__ check that the data type is the expected one, because it defines the possible operations. The `astype` method makes it possible to change the data type of a column:

In [37]:
data = (('01', True, 'a'), ('02', False, 'c'), ('03', True, 'c'))
columns = ('Some integers', 'Some booleans', 'Some strings')
df = pd.DataFrame(data=data, columns=columns, index=('first row', 'second row', 'third row'))
print(df.dtypes)
df

Some integers    object
Some booleans      bool
Some strings     object
dtype: object


,Some integers,Some booleans,Some strings
first row,01,True,a
second row,02,False,c
third row,03,True,c


In [38]:
df['Some integers'] = df['Some integers'].astype(int)
print(df.dtypes)
df

Some integers     int64
Some booleans      bool
Some strings     object
dtype: object


,Some integers,Some booleans,Some strings
first row,1,True,a
second row,2,False,c
third row,3,True,c
